<a href="https://colab.research.google.com/github/SeokcheonMoon/study_data_analytics/blob/main/docs/quests/EDA/EDA_ShoppingMallDeliveryWithDate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import seaborn as sns
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [53]:
df_shop = pd.read_csv("ShoppingMallDeliveryWithDate.csv")
df_shop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46749 entries, 0 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    46749 non-null  object 
 1   배송번호    46749 non-null  object 
 2   배송시작일   46749 non-null  object 
 3   배송완료일   46749 non-null  object 
 4   상품구매금액  46749 non-null  int64  
 5   상품번호    46661 non-null  float64
 6   수량      46749 non-null  int64  
 7   주문경로    46749 non-null  object 
 8   주문일     46749 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.2+ MB


# [목표 제안]
- 제안 2: 고객별 배송 선호도 분석

- 문제 정의: 고객별로 선호하는 배송 시간대와 요일을 분석하여 맞춤형 배송 옵션을 제공합니다.

- 배경: 개인화된 배송 서비스는 고객 경험을 개선하고 재구매율을 높일 수 있습니다.


## # 1. 접근 방법

- "고객별" 정의
  + 고객의 ID를 통합할 수 없음
  + 물품 구매에 있어 개개인이 특정시간을 두고 구매하려고 계획하지 않음.
  + => 각 row의 고객을 독립적인 개체로 판단.

- 요일별 주문건수 분석 : 주문일에서 요일데이터를 이용하여 주문이 많은 날에 이벤트 및 프로모션 등의 추가여부를 고려

  + 주문이 많은 날 - 이용자가 접근을 많이 하는것으로 판단

- 배송기간 데이터 분석 : 배송완료일과 배송시작일을 이용하여 배송기간을 분석하고 배송기간별 고객의 주문건수를 추출하여 배송기간 단축 필요성 파악

  + 어느 정도의 배송기간까지 고객의 구매여부에 영향을 줄지 파악

## # 2. 데이터 분석

### 요일별 주문건수 상관관계 분석

In [28]:
df_shop

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077.0,1,PC쇼핑몰,20181227
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42.0,2,모바일웹,20181229
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271.0,1,모바일웹,20181230
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085.0,1,모바일웹,20181231
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106.0,1,모바일웹,20181231
...,...,...,...,...,...,...,...,...,...
46744,카드,D-20171229-0000036-00,2017-12-29 오후 4:32,2018-01-02 오전 5:31,52800,445.0,1,모바일웹,20171229
46745,카드,D-20171229-0000043-00,2018-01-03 오전 9:45,2018-01-07 오전 5:38,112200,172.0,2,모바일웹,20171229
46746,카드,D-20171230-0000022-01,2018-01-02 오후 7:06,2018-01-11 오전 2:46,36000,140.0,1,네이버 페이,20171230
46747,카드,D-20171231-0000017-01,2018-01-03 오후 2:00,2018-01-06 오후 6:06,73460,184.0,1,네이버 페이,20171231


In [55]:
df_shop["주문일_datetime"] = pd.to_datetime(df_shop["주문일"], format= "%Y%m%d")
df_shop["주문일_datetime"]

0       2018-12-27
1       2018-12-29
2       2018-12-30
3       2018-12-31
4       2018-12-31
           ...    
46744   2017-12-29
46745   2017-12-29
46746   2017-12-30
46747   2017-12-31
46748   2017-12-31
Name: 주문일_datetime, Length: 46749, dtype: datetime64[ns]

In [56]:
df_shop["주문요일"] = df_shop["주문일_datetime"].dt.day_name()
df_shop["주문요일"]

0        Thursday
1        Saturday
2          Sunday
3          Monday
4          Monday
           ...   
46744      Friday
46745      Friday
46746    Saturday
46747      Sunday
46748      Sunday
Name: 주문요일, Length: 46749, dtype: object

In [31]:
df_shop["주문요일"].value_counts()

Monday       9903
Wednesday    8737
Thursday     8697
Tuesday      8404
Friday       6665
Sunday       2429
Saturday     1914
Name: 주문요일, dtype: int64

In [32]:
# 분석결과 - 상대적으로 평일에 주문이 많고 주말에 주문이 다소 적은 것을 확인할 수 있음.

# 인사이트 - (1) 평일에 이용자의 접근이 많은 것을 확인하였으니 할인 프로모션 또는 상품 구매 유도를 위한 추가 상품 마케팅 고려.
#            (2) 주말에 이용자를 늘리기 위해  주말에 구매하면 받을 수 있는 혜택 추가 고려.

# 차후에 시각화 그래프까지 만들어보자.

### 배송시간 데이터 분석

In [57]:
df_shop

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,주문일_datetime,주문요일
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077.0,1,PC쇼핑몰,20181227,2018-12-27,Thursday
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42.0,2,모바일웹,20181229,2018-12-29,Saturday
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271.0,1,모바일웹,20181230,2018-12-30,Sunday
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085.0,1,모바일웹,20181231,2018-12-31,Monday
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106.0,1,모바일웹,20181231,2018-12-31,Monday
...,...,...,...,...,...,...,...,...,...,...,...
46744,카드,D-20171229-0000036-00,2017-12-29 오후 4:32,2018-01-02 오전 5:31,52800,445.0,1,모바일웹,20171229,2017-12-29,Friday
46745,카드,D-20171229-0000043-00,2018-01-03 오전 9:45,2018-01-07 오전 5:38,112200,172.0,2,모바일웹,20171229,2017-12-29,Friday
46746,카드,D-20171230-0000022-01,2018-01-02 오후 7:06,2018-01-11 오전 2:46,36000,140.0,1,네이버 페이,20171230,2017-12-30,Saturday
46747,카드,D-20171231-0000017-01,2018-01-03 오후 2:00,2018-01-06 오후 6:06,73460,184.0,1,네이버 페이,20171231,2017-12-31,Sunday


In [40]:
df_shop["주문경로"].value_counts()

PC쇼핑몰     25695
모바일웹      18531
네이버 페이     2523
Name: 주문경로, dtype: int64

In [58]:
def convert_time(x):                                            # x -> "2019-01-11 오후 05:32" -> 2019-01-11 17:32 로 전환되는 과정의 function
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date} {time}", format="%Y-%m-%d %H:%M")

df_shop["배송시작일_datetime"] = df_shop["배송시작일"].apply(convert_time)
df_shop["배송완료일_datetime"] = df_shop["배송완료일"].apply(convert_time)

In [63]:
df_shop["배송기간"] = df_shop["배송완료일_datetime"] - df_shop["배송시작일_datetime"]
df_shop["배송기간_days"] = df_shop["배송기간"].dt.days
df_shop["배송기간_days"]

0        3
1        3
2        3
3        3
4        3
        ..
46744    3
46745    3
46746    8
46747    3
46748    3
Name: 배송기간_days, Length: 46749, dtype: int64

In [64]:
df_shop["배송기간_days"].value_counts()

3     43520
5       819
8       664
7       432
9       314
1       208
4       198
0       174
6       144
2       140
10       30
11       29
14       11
27       11
19       10
13       10
12        9
23        6
20        5
28        4
24        2
46        2
16        2
17        1
15        1
25        1
21        1
31        1
Name: 배송기간_days, dtype: int64

In [ ]:
# 분석결과 - 당연한 말이지만 배송기간이 짧을수록 주문이 많음.

# 인사이트 -
#
# 차후에 시각화 그래프까지 만들어보자.